In [1]:
import requests
import altair as alt
import numpy as np
import pandas as pd
import sys
import warnings

from collections import OrderedDict 
from datetime import datetime, timezone
from io import StringIO

%load_ext watermark
%watermark -iv -v

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

requests: 2.25.1
pandas  : 1.1.5
altair  : 4.1.0
numpy   : 1.19.2
sys     : 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]



### Load latest data

In [2]:
try:
    url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'
    r = requests.get(url, allow_redirects=True)
    df = pd.read_csv(StringIO(r.content.decode("utf-8")))[['location', 'date', 'total_vaccinations_per_hundred']]
    max_values = df.groupby('location').max('total_vaccinations_per_hundred').sort_values('total_vaccinations_per_hundred', ascending=False).reset_index(drop=False)
    display(df.sample(5))
except:
    warnings.warn('Cannot read vaccination data to pandas dataframe!')

,location,date,total_vaccinations_per_hundred
97,World,2020-12-30,0.06
30,Denmark,2020-12-27,0.09
29,Costa Rica,2020-12-24,0.00
13,Canada,2020-12-18,0.03
6,Bahrain,2020-12-28,3.23


### Plot routine

In [3]:
def plot_chart(df, cmap):
    return alt.Chart(df[df.location.isin(cmap.keys())]).mark_line(point=True, strokeWidth=2).encode(
        x       = alt.X('date:T', title='Date'),
        y       = alt.Y('total_vaccinations_per_hundred:Q', title='Vaccinated [%]'),
        color   = alt.Color('location:N', title='Country', scale=alt.Scale(
                domain = list(cmap.keys()),
                range  = list(cmap.values())
        )),    tooltip = ['location', 'date:T', 'total_vaccinations_per_hundred']
    ).configure_point(
        size=50
    ).properties(width=200*1.61, height=200, title=f'Vaccination status {datetime.now().astimezone(timezone.utc).strftime("%Y-%m-%d %H:%M")} (UTC)')

### Main countries

In [4]:
country_vs_color = OrderedDict({
    'Israel':         'lightblue',
    'Bahrain':        'yellow',
    'United Kingdom': 'purple',
    'United States':  'blue',
    'Canada':         'lightgray',
    'China':          'red',
    'Russia':         'brown',
    'European Union': 'orange',
    'Japan':          'gray'
})

In [5]:
cmap = OrderedDict({k:country_vs_color[k] for k in [x for x in max_values.location if x in country_vs_color.keys()]})
print('Main countries:     ', ",".join(list(cmap.keys())))
chart = plot_chart(df, cmap)
#chart.save('out/main-countries.png')

Main countries:      Israel,Bahrain,United Kingdom,United States,Canada,China,Russia


### European countries

In [6]:
country_vs_color = OrderedDict({'Portugal': 'green',
 'Denmark': 'red',
 'Lithuania': 'orange',
 'Germany': 'brown',
 'Estonia': 'lightblue',
 'Latvia': 'pink',
 'Italy': 'lightgreen',
 'Poland': 'lightgray',
 'Romania': 'yellow'})

In [7]:
cmap = OrderedDict({k:country_vs_color[k] for k in [x for x in max_values.location if x in country_vs_color.keys()]})
print('European countries: ', ",".join(list(cmap.keys())))
chart = plot_chart(df, cmap)
#chart.save('out/european-countries.png')

European countries:  Portugal,Denmark,Germany,Lithuania,Estonia,Latvia,Poland,Italy,Romania
